In [ ]:
from download_from_database import DownloadTables
import pandas as pd
import torch
import numpy as np 
from PIL import Image
from alive_progress import alive_bar
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
 

# Retrieve Data

In [5]:
credentials_location = ".gitignore/credentials_for_marketplace.yml"
downloader = DownloadTables(credentials_location)
downloader.download_table("images")
downloader.download_table("products")


data/images_table.json  is already downloaded, skipping
data/products_table.json  is already downloaded, skipping


In [2]:
product_df = pd.read_json("data/products_table_clean.json")
image_df = pd.read_json("data/images_table.json")

# Create a Pipeline for the Model

In [4]:
def get_image_category(image_name):
     
    image_row = image_df.loc[image_df["id"] == image_name]
    product_id = image_row.iloc[0]["product_id"]
    product_row = product_df.loc[product_df["id"] == product_id]
    category = product_row.iloc[0]["main_category"]
    
    return category


In [7]:
images = set(image_df["id"])
images_folder = "data/cleaned_images_64"

In [8]:
def process_image(image_location):
    image = Image.open(image_location)
    image = ToTensor()(image)
    image = torch.flatten(image)
    return image.numpy()
    

In [9]:
def convert_to_image_array(image):
    image_category = get_image_category(image)
    image_location = images_folder +"/"+ image +".jpg"
    image_array = process_image(image_location)
    return image_array, image_category
     

In [10]:
def create_dict_of_categories(df, column_to_index): 
    categories = set(df[column_to_index]) 
    categories_dict = {k: v for v, k in enumerate(categories)} 
    return categories_dict

category_numbers = create_dict_of_categories(product_df, "main_category")

In [11]:

array_size = 64*64*3

#TODO this bit is messy so fix it
images = list(images)

# n = 20000
n = len(images)
X = np.zeros((n,array_size))
y = np.zeros(n)



with alive_bar(n) as bar:
    for index in range(n):
        print("image number :", index)
        image = images[index]
        try:
            features, label = convert_to_image_array(image)
            X[index, :] = features
            y[index] = category_numbers[label]

        except:
            #TODO deal with this because it might be affecting the model
            X[index, :] = np.zeros(array_size)
            y[index] = 0

        bar()

on 0: image number : 0
on 1: image number : 1
on 2: image number : 2
on 3: image number : 3
on 4: image number : 4
on 5: image number : 5
on 6: image number : 6
on 7: image number : 7
on 8: image number : 8
on 9: image number : 9
on 10: image number : 10
on 11: image number : 11
on 12: image number : 12
on 13: image number : 13
on 14: image number : 14
on 15: image number : 15
on 16: image number : 16
on 17: image number : 17
on 18: image number : 18
on 19: image number : 19
on 20: image number : 20
on 21: image number : 21
on 22: image number : 22
on 23: image number : 23
on 24: image number : 24
on 25: image number : 25
on 26: image number : 26
on 27: image number : 27
on 28: image number : 28
on 29: image number : 29
on 30: image number : 30
on 31: image number : 31
on 32: image number : 32
on 33: image number : 33
on 34: image number : 34
on 35: image number : 35
on 36: image number : 36
on 37: image number : 37
on 38: image number : 38
on 39: image number : 39
on 40: image number 

In [16]:
y

array([ 6.,  7.,  5., ..., 12., 12., 11.])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [13]:

model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(score)

/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

0.16660055533518445
